# 0. Load Libraries

In [ ]:
import os
import json

import models
import utils
from classes import  Locatable, Satellite, Cluster, Vehicle
from drawingmap import DrawingMap

# 1. Load Data

## 1.1. Satellites

In [ ]:
satellites, df_satellites = utils.LoadingData.load_satellites(DEBUG=False)
print(f'Cantidad de satellites cargados: {len(satellites)}')

## 1.2. Customer Clusters

In [ ]:
clusters, df_clusters = utils.LoadingData.load_customer_clusters(DEBUG=False)
print(f'Cantidad de clusters cargados: {len(clusters)}')

## 1.3. Load Vehicles

In [ ]:
small_vehicle = Vehicle(id='small'
                        , type='small'
                        , capacity=12
                        , costFixed=20
                        , time_service=0.05
                        , time_fixed=0.05
                        , time_load=0.0072
                        , time_dispatch=0.625
                        , speed_line=40
                        , Tmax=12
                        , k=1.3)
large_vehicle = Vehicle(id='large'
                        , type='large'
                        , capacity=53
                        , costFixed=268
                        , time_service=0.05
                        , time_fixed=0.05
                        , time_load=0.0142
                        , time_dispatch=0.75
                        , speed_line=80
                        , Tmax=12
                        , k=1.3)

## 1.4. Load Matrixes: Distance and Durations with and without traffic

### 1.4.1. From Satellites to Clusters

In [ ]:
matrixes_from_satellites = utils.LoadingData.load_distances_duration_matrix_from_satellite()

### 1.4.2. From DC to Clusters

In [ ]:
matrixes_from_dc = utils.LoadingData.load_distances_duration_matrix_from_dc()

# 2. Drawing Map

## 2.1. La Paz with customer segments

In [ ]:
location_la_paz = (-16.501457, -68.149887)
location_DC = (-16.5354544, -68.1958506)

drawer = DrawingMap(location_la_paz)

drawer.addNodes(list_locatables=list(clusters.values()), color="blue", radius=1)
drawer.addNodes(list_locatables=list(satellites.values()), color='red', radius=3)
drawer.addMarker(location=location_DC, label='DC')

map = drawer.viewMap()
map

# 3. Generate Params

In [ ]:
periods = 10

params_gurobi = {
    'timeLimit':36000,
    'MIPGap':0.05
}

## 3.1. Average Fleet Size
#### a) From Satellite to Customer Cluster

In [ ]:
config_ = utils.ConfigDeterministic()
fleet_size_satellites = config_.calculate_avg_fleet_size_from_satellites(satellites=satellites, clusters=clusters, vehicle=small_vehicle
                                                                         ,periods=periods, distances_linehaul=matrixes_from_satellites['distance'])

#### b) From DC to Customer Cluster

In [ ]:
fleet_size_dc = config_.calculate_avg_fleet_size_from_dc(clusters=clusters, vehicle=large_vehicle, periods=periods
                                                         , distances_linehaul=matrixes_from_dc['distance'])

## 3.3. Cost *$c_{sk}^{t}$* and *$g_{k}{t}$*

In [ ]:
# determinar cost of shipping

In [ ]:
def cost_satellite_cluster_period(satellites: list[Satellite],
                                  clusters: list[Cluster],
                                  cost_shipping: dict[(str, str), float],
                                  vehicle_small: Vehicle,
                                  periods: int,
                                  vehicles_required: dict[str, dict]) -> dict[(str,str,int), float]:
    costs = {}
    for t in range(periods):
        for k in clusters:
            for s in satellites:
                costs[(s.id, k.id, t)] = (s.costSourcing*k.demand[t]) + (cost_shipping[(s.id, k.id)]*k.demand[t]) + \
                                         (vehicle_small.costFixed*vehicles_required['satellite'][(s.id, k.id, t)])
    return costs

def cost_dc_cluster_period(clusters: list[Cluster],
                           cost_shipping: dict[str, float],
                           vehicle_large: Vehicle,
                           periods: int,
                           vehicles_required: dict[str, dict]) -> dict[(str,int), float]:
    costs = {}
    for t in range(periods):
        for k in clusters:
            costs[(k.id, t)] = (cost_shipping[k.id] * k.demand[t]) + (vehicle_large.costFixed * vehicles_required['dc'][(k.id, t)])
    return costs

In [ ]:
cost_C = cost_satellite_cluster_period(satellites, clusters
                                       ,cost_shipping=None
                                       ,vehicle_small=small_vehicle
                                       ,periods=periods
                                       ,vehicles_required=fleet_size_satellites)

cost_G = cost_dc_cluster_period(clusters
                                ,cost_shipping=None
                                ,vehicle_large=large_vehicle
                                ,periods=periods
                                ,vehicles_required=fleet_size_dc)

# 4. Model Deterministic

In [ ]:
model_ = models.ModelDeterministic(periods=periods)

In [ ]:
model_.build(satellites=satellites
             ,clusters=clusters
             ,vehicles_required={'small':fleet_size_satellites, 'large':fleet_size_dc}
             ,costs={'satellite':cost_C, 'dc':cost_G})

model_.setParams(params=params_gurobi)

In [ ]:
print(model_.optimizeModel())